# Simple LeNet for MNIST classification

Inspired by Yann LeCunns LeNet-5 (Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proceedings of the IEEE, 1998).

In [1]:
using Knet
using NNHelferlein

### Get MNIST data from MLDatasets:

In [3]:
xtrn, ytrn, xtst, ytst = dataset_mnist()
@show dtrn = minibatch(xtrn, ytrn, 128; xsize = (28,28,1,:))
@show dtst = minibatch(xtst, ytst, 128; xsize = (28,28,1,:));

dtrn = minibatch(xtrn, ytrn, 128; xsize = (28, 28, 1, :)) = 468-element Knet.Train20.Data{Tuple{CuArray{Float32}, Array{Int64}}}
dtst = minibatch(xtst, ytst, 128; xsize = (28, 28, 1, :)) = 78-element Knet.Train20.Data{Tuple{CuArray{Float32}, Array{Int64}}}


### Define LeNet with NNHelferlein types:

In [4]:
lenet = Classifier(Conv(5,5,1,20),       
                Pool(),
                Conv(5,5,20,50),
                Pool(),
                Flat(),
                Dense(800,512), 
                Dense(512,10, actf=identity)
        )

Classifier(Any[Conv(P(CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}(5,5,1,20)), P(CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}(1,1,20,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Conv(P(CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}(5,5,20,50)), P(CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}(1,1,50,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Flat(), Dense(P(CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}(512,800)), P(CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}(512)), Knet.Ops20.sigm), Dense(P(CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}(10,512)), P(CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}(10)), identity)])

In [5]:
print_network(lenet)

NNHelferlein neural network summary:
Classifier with 7 layers,                                       440812 params
Details:
 
    Conv layer 1 → 20 (5,5) with relu,                             520 params
    Pool layer,                                                      0 params
    Conv layer 20 → 50 (5,5) with relu,                          25050 params
    Pool layer,                                                      0 params
    Flat layer,                                                      0 params
    Dense layer 800 → 512 with sigm,                            410112 params
    Dense layer 512 → 10 with identity,                           5130 params
 
Total number of layers: 7
Total number of parameters: 440812


7

### Train with Tensorboard logger:

In [6]:
tb_train!(lenet, Adam, dtrn, epochs=2, split=0.8,
        acc_fun=accuracy,
        eval_size=0.2, eval_freq=5, mb_loss_freq=100, 
        tb_name="example_run", tb_text="NNHelferlein example")

println("Test loss:           $(lenet(dtst))")
println("Test accuracy:       $(accuracy(lenet, data=dtst))");

Splitting dataset for training (80%) and validation (20%).
Training 2 epochs with 374 minibatches/epoch and 94 validation mbs.
Evaluation is performed every 75 minibatches with 19 mbs.
Watch the progress with TensorBoard at:
/home/andreas/Documents/Projekte/2022-NNHelferlein_KnetML/NNHelferlein/examples/logs/example_run/2022-05-14T08-39-33


Progress: 100%|█████████████████████████████████████████| Time: 0:00:27


Training finished with:
Training loss:       0.060302062710886575
Training accuracy:   0.9816176470588235
Validation loss:     0.06592319967185563
Validation accuracy: 0.9793051861702128
Test loss:           0.06389004
Test accuracy:       0.9805689102564102


Tensorboard output:    
<img src="assets/10-mnist-tb.png">